In [ ]:
from IPython.display import clear_output
!pip install pennylane

clear_output()

In [ ]:
import numpy as np
import pandas as pd
import pennylane as qml
import scipy
import torch
import torchvision
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import jax
from torchvision import datasets
import torchvision.transforms.v2 as transforms
from PIL import Image
from torchvision.datasets.vision import VisionDataset
import matplotlib.pyplot as plt

In [ ]:
num_qubits = 3
num_layers = 3

In [ ]:
dev = qml.device("default.qubit", wires=num_qubits)


@qml.qnode(dev, interface='torch')
def circuit(inputs, params):
    # Encoding of classical input values using RY gates
    for j in range(num_qubits):
        qml.RY(np.pi * inputs[j], wires=j)

    # Strongly entangled paramterised quantum circuit
    qml.StronglyEntanglingLayers(weights=params, wires=range(num_qubits))     #, imprimitive=qml.ops.CZ

    # Measurement producing 1 classical output value
    return qml.expval(qml.PauliZ(num_qubits-1))

@qml.qnode(dev, interface='torch')
def drc_circuit(inputs, input_params, weights):   #, output_params
    # Encoding of classical input values using RY gates
    drc_layers = input_params.shape[0]

    inputs = torch.reshape(inputs, (drc_layers, int(inputs.shape[0]/drc_layers)))
    for layer in range(drc_layers):
      for j in range(num_qubits):
        qml.RY(input_params[layer,j] * inputs[layer,j], wires=j)  #np.pi

      qml.StronglyEntanglingLayers(weights=weights[layer,:,:].reshape((1,weights.shape[1],weights.shape[2])), wires=range(num_qubits))    #, imprimitive=qml.ops.CZ

    # Measurement producing 1 classical output value
    return qml.expval(qml.PauliZ(num_qubits-1))

In [ ]:
class QuantumConvolution(torch.nn.Module):
  def __init__(self, input_size, in_filters, out_filters, kernel_size, stride, kernel, layers_per_circuit):
    super(QuantumConvolution, self).__init__()
    self.in_filters = in_filters
    self.out_filters = out_filters
    self.kernel = kernel
    self.kernel_size = kernel_size
    self.stride = stride
    self.lpc = layers_per_circuit
    self.iter = int(np.ceil(1 + (input_size[0] - self.kernel_size[0])/self.stride))
    drc_layers = int((kernel_size[0]*kernel_size[1]) / num_qubits)
    self.params_per_kernel_shape = qml.StronglyEntanglingLayers.shape(n_layers=drc_layers, n_wires=num_qubits)   #num_layers

    for o_f in range(self.out_filters):
      for i_f in range(self.in_filters):
        self.add_module('torch_window_'+str(o_f)+str(i_f), qml.qnn.TorchLayer(drc_circuit, {'input_params': (drc_layers, num_qubits), 'weights':self.params_per_kernel_shape}, init_method = torch.nn.init.normal_))

  def forward(self, inputs):
    output = torch.zeros((inputs.shape[0], self.out_filters, self.iter, self.iter))

    for o_f in range(self.out_filters):
      for i_f in range(self.in_filters):
        for l in range(self.iter):
          for b in range(self.iter):

            flattened_inputs_window = torch.nn.Flatten()(inputs[:, i_f, l*self.stride : l*self.stride + self.kernel_size[0], b*self.stride : b*self.stride + self.kernel_size[0]])
            out_i_f = torch.vmap(self.get_submodule('torch_window_'+str(o_f)+str(i_f)))(flattened_inputs_window)
            output[:,o_f,l,b] += out_i_f

    return output

In [ ]:
class QuantumModel(torch.nn.Module):
  def __init__(self):
    super(QuantumModel, self).__init__()

    # self.layer_1 = QuantumConvolution(input_size=(28,28), filters=3, kernel_size=(2,2), stride=2, kernel=circuit, layers_per_circuit=1)
    # self.layer_2 = QuantumConvolution(input_size=(14,14), filters=3, kernel_size=(2,2), stride=2, kernel=circuit, layers_per_circuit=num_layers)
    self.layer_3 = QuantumConvolution(input_size=(18+1,18+1), in_filters=1, out_filters=3, kernel_size=(3,3), stride=2, kernel=circuit, layers_per_circuit=num_layers)
    self.layer_4 = QuantumConvolution(input_size=(9,9), in_filters=3, out_filters=1, kernel_size=(3,3), stride=2, kernel=circuit, layers_per_circuit=num_layers)
    # self.layer_5 = QuantumConvolution(input_size=(4+1,4+1), in_filters=2, out_filters=1, kernel_size=(3,3), stride=2, kernel=circuit, layers_per_circuit=1)
    # self.layer_6 = QuantumConvolution(input_size=(2,2), filters=3, kernel_size=(2,2), stride=1, kernel=circuit, layers_per_circuit=1)
    self.layer_7 = torch.nn.Flatten()
    self.layer_8 = torch.nn.Linear(16,2)

  def forward(self, x):
    x = torch.nn.functional.pad(x, (0,1,0,1), 'constant', value = 0)
    x = self.layer_3(x)
    x = self.layer_4(x)
    x = self.layer_7(x)
    x = self.layer_8(x)
    x = torch.nn.functional.normalize(x)

    return x

In [ ]:
qm = QuantumModel()
def count_parameters(model):
  model_parameters = filter(lambda p: p.requires_grad, model.parameters())
  params = sum([np.prod(p.size()) for p in model_parameters])
  return params

count_parameters(qm)

In [ ]:
class TwoCropTransform:
    """Create two crops of the same image"""
    def __init__(self, transform):
        self.transform = transform

    def __call__(self, x):
        return [self.transform(x), self.transform(x)]

In [ ]:
class SupConLoss(nn.Module):
    def __init__(self, temperature=0.07, contrast_mode='all',
                 base_temperature=0.07):
        super(SupConLoss, self).__init__()
        self.temperature = temperature
        self.contrast_mode = contrast_mode
        self.base_temperature = base_temperature

    def calculate_fidelity(self, feature_1, feature_2):
      swap_test = SwapTest(feature_1, feature_2)
      fidelity = torch.sub(torch.mul(swap_test[:,0].reshape(feature_1.shape[0], feature_2.shape[0]), 2), 1)
      return fidelity

    def matrix_mul(self, mat1, mat2):
      return torch.matmul(mat1, mat2.T)

    def dot_prod(self, v1, v2):
      return torch.dot(v1,v2)

    def forward(self, features, labels=None, mask=None):

        if len(features.shape) < 3:
            raise ValueError('`features` needs to be [bsz, n_views, ...],'
                             'at least 3 dimensions are required')
        if len(features.shape) > 3:
            features = features.view(features.shape[0], features.shape[1], -1)

        batch_size = features.shape[0]
        if labels is not None and mask is not None:
            raise ValueError('Cannot define both `labels` and `mask`')
        elif labels is None and mask is None:
            mask = torch.eye(batch_size, dtype=torch.float32) #.to(device)
        elif labels is not None:
            labels = labels.contiguous().view(-1, 1)
            if labels.shape[0] != batch_size:
                raise ValueError('Num of labels does not match num of features')
            mask = torch.eq(labels, labels.T).float() #.to(device)
        else:
            mask = mask.float()

        contrast_count = features.shape[1]
        contrast_feature = torch.cat(torch.unbind(features, dim=1), dim=0)
        if self.contrast_mode == 'one':
            anchor_feature = features[:, 0]
            anchor_count = 1
        elif self.contrast_mode == 'all':
            anchor_feature = contrast_feature
            anchor_count = contrast_count
        else:
            raise ValueError('Unknown mode: {}'.format(self.contrast_mode))


        anchor_dot_contrast = torch.div(
            (torch.vmap(torch.vmap(self.dot_prod, in_dims=(None,0)), in_dims=(0,None)))(anchor_feature, contrast_feature),
            self.temperature
            )
        logits_max, _ = torch.max(anchor_dot_contrast, dim=1, keepdim=True)
        logits = anchor_dot_contrast - logits_max.detach()

        # tile mask
        mask = mask.repeat(anchor_count, contrast_count)

        # mask-out self-contrast cases
        logits_mask = torch.scatter(
            torch.ones_like(mask),
            1,
            torch.arange(batch_size * anchor_count).view(-1, 1), #.to(device),
            0
        )
        mask = mask * logits_mask

        # compute log_prob
        exp_logits = torch.exp(logits) * logits_mask
        log_prob = logits - torch.log(exp_logits.sum(1, keepdim=True))

        # compute mean of log-likelihood over positive
        mean_log_prob_pos = (mask * log_prob).sum(1) / mask.sum(1)

        # loss
        loss = - (self.temperature / self.base_temperature) * mean_log_prob_pos
        loss = loss.view(anchor_count, batch_size).mean()

        return loss

In [ ]:
from collections import defaultdict

class MetricMonitor:
    def __init__(self, float_precision=4):
        self.float_precision = float_precision
        self.reset()

    def reset(self):
        self.metrics = defaultdict(lambda: {"val": 0, "count": 0, "avg": 0})

    def update(self, metric_name, val):
        metric = self.metrics[metric_name]

        metric["val"] += val
        metric["count"] += 1
        metric["avg"] = metric["val"] / metric["count"]

    def __str__(self):
        return " | ".join(
            [
                "{metric_name}: {avg:.{float_precision}f}".format(
                    metric_name=metric_name, avg=metric["avg"], float_precision=self.float_precision
                )
                for (metric_name, metric) in self.metrics.items()
            ]
        )

In [ ]:
def quantum_state_fidelity(state1, state2):
  return 1 - torch.abs(torch.dot(torch.conj(state1), state2))**2

In [ ]:
def lalign(x, y, alpha=2):
  return (x - y).norm(dim=1).pow(alpha).mean()

def lunif(x, t=2):
  sq_pdist = torch.pdist(x, p=2).pow(2)
  return sq_pdist.mul(-t).exp().mean().log()

In [ ]:
def pretraining(epoch, model, contrastive_loader, optimizer, criterion, method='SimCLR', lama=None, lamu=None, alpha=None, t=None):  #SupCon
    "Contrastive pre-training over an epoch"
    metric_monitor = MetricMonitor()
    for batch_idx, (data,labels) in enumerate(contrastive_loader):
        data = torch.cat([data[0], data[1]], dim=0)
        if torch.cuda.is_available():
            data,labels = data.cuda(), labels.cuda()
        data, labels = torch.autograd.Variable(data,False), torch.autograd.Variable(labels)
        bsz = labels.shape[0]
        features = model(data)
        f1, f2 = torch.split(features, [bsz, bsz], dim=0)
        features = torch.cat([f1.unsqueeze(1), f2.unsqueeze(1)], dim=1)
        if method == 'SupCon':
            loss = criterion(features, labels)
        elif method == 'SimCLR':
            loss = criterion(features)
            la = lalign(features[:,0,:], features[:,1,:])
            lux = lunif(features[:,0,:])
            luy = lunif(features[:,1,:])
            print('lalign', la)
            print('luniform', (lux + luy) / 2)
        elif method == 'AlignAndUniform':
            # print(features[:,0,:].shape)
            la = lalign(features[:,0,:], features[:,1,:], alpha)
            lux = lunif(features[:,0,:], t)
            luy = lunif(features[:,1,:], t)
            print('lalign', la)
            print('luniform', (lux + luy) / 2)
            loss = lama * la + lamu * (lux + luy) / 2
        else:
            raise ValueError('contrastive method not supported: {}'.format(method))
        metric_monitor.update("Loss", loss.item())
        metric_monitor.update("Learning Rate", optimizer.param_groups[0]['lr'])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print("[Epoch: {epoch:03d}] Contrastive Pre-train | {metric_monitor}".format(epoch=epoch, metric_monitor=metric_monitor))
    return metric_monitor.metrics['Loss']['avg'], metric_monitor.metrics['Learning Rate']['avg']

In [ ]:
class Z2_Normalization(torch.nn.Module):
  def __init__(self):
    super(Z2_Normalization, self).__init__()

  def forward(self, img):
    if len(img.shape) > 2:
      mean = torch.Tensor([img[0].mean(), img[1].mean(), img[2].mean()])
      std = torch.Tensor([img[0].std(), img[1].std(), img[2].std()])
      return torchvision.transforms.functional.normalize(img, mean, std, inplace=True)
    else:
      return torchvision.transforms.functional.normalize(img, img.mean(), img.std(), inplace=True)

In [ ]:
# def main():

num_epochs = 50
batch_size = 256  #1280
use_early_stopping = True
use_scheduler = False


contrastive_transform = transforms.Compose([
                                    transforms.RandomHorizontalFlip(p=0.5),
                                    transforms.RandomVerticalFlip(p=0.5),
                                    # transforms.RandomRotation([0,180]),
                                    transforms.ToImage(),
                                    transforms.ToDtype(torch.float32, scale=True),
                                    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)),
                                    # Z2_Normalization(),
                                    ])


contrastive_set = datasets.MNIST('./data', download=True, train=True, transform=TwoCropTransform(contrastive_transform))

inds = []
for index,i in enumerate(contrastive_set.targets):
  if i == 0 or i == 8:
    inds.append(index)

contrastive_set.data = contrastive_set.data[inds]
contrastive_set.targets = contrastive_set.targets[inds]

# Resizing images to size 8x8
contrastive_set.data = transforms.Resize(18)(contrastive_set.data)
print(contrastive_set.data.shape)
# Define the desired subset size
subset_size = 1024 #5120

# Create a subset of the dataset
subset_indices = torch.randperm(len(contrastive_set), generator=torch.manual_seed(0))[:subset_size]
subset_con = torch.utils.data.Subset(contrastive_set, subset_indices)

contrastive_loader = torch.utils.data.DataLoader(subset_con, batch_size=batch_size, shuffle=True)

(5000, 125, 125, 3)


In [ ]:
# Part 1
model = QuantumModel()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08)  #, weight_decay=1e-3
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=num_epochs/5, gamma=0.9)

contrastive_loss, contrastive_lr = [], []

for epoch in range(1, num_epochs+1):
  loss, lr = pretraining(epoch, model, contrastive_loader, optimizer, criterion, method='AlignAndUniform', lama=1, lamu=0.9, alpha=2, t=2)  #SupCon #SimCLR #
  if use_scheduler:
    scheduler.step()
  print(criterion.temperature)
  contrastive_loss.append(loss)
  contrastive_lr.append(lr)

# save_model(model, optimizer, num_epochs, save_file)

plt.plot(range(1,len(contrastive_lr)+1),contrastive_lr, color='b', label = 'learning rate')
plt.legend(), plt.ylabel('loss'), plt.xlabel('epochs'), plt.title('Learning Rate'), plt.show()

plt.plot(range(1,len(contrastive_loss)+1),contrastive_loss, color='b', label = 'loss')
plt.legend(), plt.ylabel('loss'), plt.xlabel('epochs'), plt.title('Loss'), plt.show()

In [ ]:
model.training = False
cls_epochs = 1000

cls_data_set = datasets.MNIST('./data', download=True, train=True,
                              transform=transforms.Compose([ #transforms.ToTensor(),
                                                             transforms.ToImage(),
                                                             transforms.ToDtype(torch.float32, scale=True),
                                                             transforms.Normalize((0.5,), (0.5,)),]))

cls_data_set.data = transforms.Resize(18)(cls_data_set.data)
cls_data_set.data = cls_data_set.data[inds]
cls_data_set.targets = cls_data_set.targets[inds]
cls_data_set.data = cls_data_set.data[subset_indices]         #[:1000]
cls_data_set.targets = cls_data_set.targets[subset_indices]    #[:1000]

targets = []
for i in cls_data_set.targets:
  if i == 0:
    targets.append(0)
  else:
    targets.append(1)

a = []
for i, (data, label) in enumerate(cls_data_set):
  a.append(data)

test_set_size = 250
cls_test_set = datasets.MNIST('./data', download=True, train=False,
                              transform=transforms.Compose([ #transforms.ToTensor(),
                                                             transforms.ToImage(),
                                                             transforms.ToDtype(torch.float32, scale=True),
                                                             transforms.Normalize((0.5,), (0.5,)),]))

testinds = []
for index,i in enumerate(cls_test_set.targets):
  if i == 0 or i == 8:
    testinds.append(index)

cls_test_set.data = transforms.Resize(18)(cls_test_set.data)
cls_test_set.data = cls_test_set.data[testinds]
cls_test_set.targets = cls_test_set.targets[testinds]
cls_test_set.data = cls_test_set.data[:test_set_size]
cls_test_set.targets = cls_test_set.targets[:test_set_size]

testtargets = []
for i in cls_test_set.targets:
  if i == 0:
    testtargets.append(0)
  else:
    testtargets.append(1)

b = []
for i, (data, label) in enumerate(cls_test_set):
  b.append(data)


classifier = torch.nn.Sequential(
    torch.nn.Linear(2,32),
    torch.nn.BatchNorm1d(32),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(32,1),
    torch.nn.Sigmoid()
)


# random_model = QuantumModel()
# cls_train_data = random_model(torch.stack(a))
cls_train_data = model(torch.autograd.Variable(torch.stack(a), False))   #torch.squeeze(torch.stack(a), 1)
print(cls_train_data.shape)

cls_test_data = model(torch.autograd.Variable(torch.stack(b), False))   #torch.squeeze(torch.stack(a), 1)
print(cls_test_data.shape)

targets = torch.unsqueeze(torch.Tensor(targets), 1)
cls_train_data, targets = torch.autograd.Variable(cls_train_data,False), torch.autograd.Variable(torch.Tensor(targets)).to(torch.float32)

testtargets = torch.unsqueeze(torch.Tensor(testtargets), 1)
cls_test_data, testtargets = torch.autograd.Variable(cls_test_data,False), torch.autograd.Variable(torch.Tensor(testtargets)).to(torch.float32)


def train_classifier(cls_epochs, classifier, cls_train_data, labels, subset_size, batch_size, cls_test_data, testlabels):
  optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08)
  cls_loss = []
  cls_accuracy = []
  test_cls_loss = []
  test_cls_accuracy = []

  for ce in range(cls_epochs):

    correct = 0
    total = 0
    t_correct = 0
    t_total = 0

    optimizer.zero_grad()
    outputs = classifier(cls_train_data)
    loss = torch.nn.BCELoss()(outputs,labels)
    loss.backward()
    optimizer.step()
    predicted = np.round(outputs.detach().numpy())
    total += labels.size(0)
    correct += np.sum(torch.eq(torch.Tensor(predicted), labels).detach().numpy())
    accuracy = 100 * correct / total
    cls_loss.append(loss.detach().numpy())
    cls_accuracy.append(accuracy)
    testoutputs = classifier(cls_test_data)
    testloss = torch.nn.BCELoss()(testoutputs, testlabels)
    testpredicted = np.round(testoutputs.detach().numpy())
    t_total += testlabels.size(0)
    t_correct += np.sum(torch.eq(torch.Tensor(testpredicted), testlabels).detach().numpy())
    testaccuracy = 100 * t_correct / t_total
    test_cls_loss.append(testloss.detach().numpy())  #np.mean(e_loss)
    test_cls_accuracy.append(testaccuracy)
    print(f'Epochs : {ce} ; Loss : {loss.detach().numpy()} ; Accuracy : {accuracy} ; Test Loss : {testloss} ; Test accuracy : {testaccuracy}' )   #np.mean(e_loss)

  return cls_loss, cls_accuracy, test_cls_loss, test_cls_accuracy


cls_loss, cls_accuracy, test_cls_loss, test_cls_accuracy = train_classifier(cls_epochs, classifier, cls_train_data, targets, subset_size, batch_size, cls_test_data, testtargets)

In [ ]:
plt.plot(cls_loss, label='Train loss')
plt.plot(test_cls_loss, label='Test loss')
plt.xlabel('Epochs')
plt.ylabel('BCE loss')
plt.legend()

In [ ]:
plt.plot(cls_accuracy, label='Train accuracy')
plt.plot(test_cls_accuracy, label='Test accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

In [ ]:
from sklearn.neighbors import KernelDensity

v = []
for i, (data, label) in enumerate(cls_data_set):
  v.append(data)

# randommodel = QuantumModel()
# vdata = randommodel(torch.stack(a)).detach().numpy()
vdata = model(torch.autograd.Variable(torch.stack(v), False)).detach().numpy()

In [ ]:
kde = KernelDensity(kernel='gaussian', bandwidth=0.01).fit(vdata)
gv = kde.score_samples(vdata)

gvs = kde.sample(1024)
plt.scatter(gvs[:,0], gvs[:,1])

In [ ]:
plt.scatter(vdata[:,0], vdata[:,1])
plt.xticks(np.arange(-1,1,0.1))
plt.yticks(np.arange(-1,1,0.1))
plt.show()

In [ ]:
class_0 = np.array([[0,0]])
class_1 = np.array([[0,0]])
for i,t in enumerate(cls_data_set.targets):
  if t == 8:
    class_1 = np.append(class_1, [vdata[i]],axis=0)
  else:
    class_0 = np.append(class_0, [vdata[i]],axis=0)

In [ ]:
plt.scatter(class_1[1:,0],class_1[1:,1])
plt.xticks(np.arange(-1,1,0.1))
plt.yticks(np.arange(-1,1,0.1))
plt.show()

In [ ]:
plt.scatter(class_0[1:,0],class_0[1:,1])
plt.xticks(np.arange(-1,1,0.1))
plt.yticks(np.arange(-1,1,0.1))
plt.show()